In [1]:
from ultralytics import YOLO
import cv2
import cvzone
import math

In [2]:
from sort import *

/Users/ngoni/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
model = YOLO('.../yolov8n.pt')
classNames = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
              'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 
              'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 
              'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 
              'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 
              'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 
              'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 
              'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 
              'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 
              'teddy bear', 'hair drier', 'toothbrush']


In [ ]:
# C

In [6]:
# Import necessary libraries
import cv2
import cvzone
from sort import Sort  # Import the SORT (Simple Online and Realtime Tracking) algorithm
import numpy as np
import math

# Create a VideoCapture object with the specified video file
cap = cv2.VideoCapture('../Videos/cars2.mp4')

# Set the width of VideoCapture
cap.set(3, 320)

# Initialize the SORT tracker with specified parameters
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

# Define the limits for counting vehicles
#This is a line with the coordinates x1,y1,x2,y2
limits = [200, 800, 4500, 800]

# List to store the unique IDs of detected vehicles
totalCount = []

# Main loop for video processing
while True:
    # Read a frame from the webcam
    success, img = cap.read()

    # Overlay additional graphics on the image
    imgGraphics = cv2.imread('graphics.png', cv2.IMREAD_UNCHANGED)
    cvzone.overlayPNG(img, imgGraphics, (0, 0))

    # Use YOLO model to detect objects in the frame
    results = model(img, stream=True)

    # Create an array to store detections
    detections = np.empty((0, 5))

    # Iterate over the detected objects
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Extract coordinates of the bounding box
            x1, y1, x2, y2 = box.xyxy[0]

            # Convert coordinates from tensor type to int type
            x1, x2, y1, y2 = int(x1), int(x2), int(y1), int(y2)

            # Debugging print
            print(x1, y1, x2, y2)

            # Bounding box information
            bbox = x1, y1, x2 - x1, y2 - y1

            # Confidence value rounded up to 2 decimal places
            conf = math.ceil((box.conf[0] * 100)) / 100

            # Class index
            cls = int(box.cls[0])

            # Check if the detected object is a vehicle with sufficient confidence
            currentClass = classNames[cls]
            if currentClass in ["car", "bus", "truck", "motorbike"] and conf > 0.25:
                # Append the current detection to the array
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))
                
                cvzone.putTextRect(img, f'{currentClass} {conf}', (max(0, x2-240), max(35, y1)),
                                   scale=3, thickness=5, offset=10, colorR=(0, 255, 0))
                #cvzone.cornerRect(img, (x1, y1, x2-x1, y2-y1), l=9, rt=5, colorR=(0, 0, 255))

    # Update the SORT tracker with the detected objects
    resultsTracker = tracker.update(detections)

    # Draw counting line on the image
    cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 5)

    # Iterate over the tracker results
    for result in resultsTracker:
        x1, y1, x2, y2, Id = result
        x1, x2, y1, y2 = int(x1), int(x2), int(y1), int(y2)

        # Debugging print
        print(result)

        # Draw a rectangle around the detected object
        cvzone.cornerRect(img, bbox, l=9, rt=2, colorR=(255, 0, 255))
        # Put the unique ID on the image
        cvzone.putTextRect(img, "ID: {}".format(int(Id)), (max(0, x1), max(35, y1)), scale=3, offset=10, thickness=3)

        # Calculate the center coordinates of the bounding box
        cx, cy = x1 + (x2 - x1) // 2, y1 + (y2 - y1) // 2

        # Draw a circle at the center of the bounding box
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

        # Check if the center coordinates are within the counting limits
        if limits[0] < cx < limits[2] and limits[1] - 20 < cy < limits[3] + 20:
            # If the ID is not already in the totalCount list, add it
            if totalCount.count(Id) == 0:
                totalCount.append(Id)
                # Change the color of the counting line to green
                cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5)

    # Display the count of detected vehicles on the image
    cvzone.putTextRect(img, "Count: {}".format(len(totalCount)), (50, 50), scale=2, offset=10, thickness=3)

    # Display the image in a window
    window_name = "Image"
    cv2.imshow(window_name, img)
    cv2.moveWindow(window_name, 0, 0)

    # Wait for a key event
    cv2.waitKey(2)



2639 0 3033 207
1788 180 2274 633
1784 180 2274 633
0: 384x640 2 cars, 1 truck, 78.9ms
Speed: 3.7ms preprocess, 78.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[       1784         180        2274         633          18]
[       1788         180        2274         633          17]
[       2639           0        3033         207          16]

2640 2 3034 215
1775 192 2267 650
1777 191 2267 648
0: 384x640 2 cars, 1 truck, 82.8ms
Speed: 3.5ms preprocess, 82.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1776.5      191.48      2267.5      647.52          18]
[     1775.1      191.93      2266.9      650.07          17]
[     2638.6      2.7285      3035.4      214.27          16]

2642 1 3042 222
1771 198 2265 663
1773 200 2265 663
0: 384x640 2 cars, 1 truck, 73.9ms
Speed: 4.7ms preprocess, 73.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1771.9      201.15      2265.6      661.96          18]
[     1769.

Speed: 1.8ms preprocess, 66.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[       1878      7.0721      2268.2      134.63          20]
[     2664.5      10.918      3189.3      505.72          19]
[     1419.5      572.38      2086.6      1184.7          18]

1883 0 2262 145
1382 591 2069 1221
2678 6 3183 523
2676 3 3179 526
0: 384x640 3 cars, 1 truck, 64.2ms
Speed: 1.7ms preprocess, 64.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1873.8      5.6902      2271.5      141.11          20]
[     2667.6      12.089      3192.3      517.61          19]
[     1392.1      592.53      2069.3        1214          18]

1367 618 2056 1244
1884 0 2264 160
2674 3 3188 538
2667 1 3183 541
0: 384x640 3 cars, 1 truck, 67.9ms
Speed: 1.8ms preprocess, 67.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
[     1871.3      6.6188      2275.7      151.89          20]
[     2667.4      11.474      3195.3      530.51          19]
[      

Speed: 2.1ms preprocess, 70.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[       1816      6.9997      2290.8      328.56          22]
[     2608.2      184.65      3222.2      867.34          21]
[     1818.7      10.475      2287.3      323.82          20]
[     2610.1      184.38      3226.4       865.4          19]
[     778.72      1078.6      1672.4      1891.3          18]

707 1110 1642 1954
2583 199 3232 894
1808 0 2273 350
1809 0 2277 352
2556 205 3230 892
1231 1307 1385 1440
0: 384x640 1 person, 3 cars, 2 trucks, 68.3ms
Speed: 2.6ms preprocess, 68.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1803.7      7.1553      2284.3      341.73          22]
[     2594.8      198.79      3224.4      891.63          21]
[     1808.4      10.916      2283.6      338.53          20]
[     2582.9      201.18      3220.2      889.68          19]
[     727.19      1113.6      1637.8      1940.1          18]

654 1142 1607 2008
2571 224 324

Speed: 1.8ms preprocess, 65.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1627.4      85.751      2203.9      669.96          22]
[     2310.1      607.64      3144.3        1493          21]

1618 104 2194 697
1619 104 2191 694
2258 631 3141 1545
2758 877 2932 996
2255 631 3142 1545
0: 384x640 1 person, 2 cars, 2 trucks, 64.4ms
Speed: 1.8ms preprocess, 64.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1616.2      100.72      2197.8       692.6          22]
[     2278.8      634.29      3132.7      1534.4          21]

2253 661 3126 1582
1604 125 2186 724
2748 923 2897 1037
1603 127 2187 726
0: 384x640 1 person, 1 car, 1 bus, 1 truck, 65.9ms
Speed: 2.0ms preprocess, 65.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1603.1      120.86      2190.5      719.93          22]
[     2258.9      662.16        3124      1574.3          21]
[     1602.6      124.17      2190.4      713.44          20]

2211 695 

[     1117.6      571.52      1913.3      1375.3          22]
[     1406.7      1591.3      2646.1      2213.4          19]

1375 1733 2559 2145
1064 598 1886 1430
1064 597 1886 1431
1379 1731 2564 2148
0: 384x640 2 cars, 2 trucks, 66.0ms
Speed: 2.4ms preprocess, 66.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1078.7      600.36      1886.5      1416.9          22]
[       1362      1660.1      2600.2      2212.7          21]
[     1354.1      1657.4      2600.4      2216.1          19]

1021 615 1857 1469
1324 1807 2495 2143
0: 384x640 1 car, 1 truck, 65.3ms
Speed: 2.1ms preprocess, 65.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1036.9      623.51      1857.6      1455.4          22]
[     1299.3      1733.3      2547.2      2210.9          21]

990 650 1832 1512
1291 1883 2463 2145
0: 384x640 1 car, 1 truck, 66.8ms
Speed: 2.0ms preprocess, 66.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[     1000.5 

KeyboardInterrupt: 